In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json

classes = []
input_dir = "xml_label_dir" #변경
output_dir = "txt_label_dir" #새로 만들어질 txt label 파일 명 (변경)
image_dir = "images_dir" #변경

os.mkdir(output_dir) # txt label 파일 생성

# yolo에 맞게 bbox설정
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


# xml 파일들 리스트로 저장
files = glob.glob(os.path.join(input_dir, '*.xml'))

# label.xml 파일들이 images 파일들과 모두 매치되는지 먼저 확인
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))